In [1]:

import numpy as np
import pandas as pd
import datetime
import glob
#from tqdm.auto import tqdm
import os
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#from dash import Dash, html, dcc
import tkinter
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

import warnings
warnings.filterwarnings("ignore")

exit_days = 60

In [2]:
import configparser

config = configparser.ConfigParser()
config.read('strategy_config.ini')

strategy_path = config['base']['path']
file_type = config['base']['file_type']
strategy_report_paths = glob.glob(strategy_path + '*' + file_type)

In [3]:

def stock_id_preprocess(stock_id):
    return stock_id[-4-4:-4]

def preprocess(daily_df):
    stock_ids = daily_df['股票代號'].astype('str')
    date_indexes = daily_df.iloc[:,2:].columns
    date_indexes = [datetime.strptime(date_index[:8], "%Y%m%d") for date_index in date_indexes]
    daily_df = daily_df.T.iloc[2:, :]
    daily_df.columns = stock_ids
    daily_df.index = pd.to_datetime(date_indexes)
    return daily_df


In [18]:
def preprocess_tradingRecords(strategy_report_path):
    tradingRecords = pd.read_excel(strategy_report_path, sheet_name='交易分析')[['商品名稱', '進場時間']]
    tradingRecords['進場時間'] = pd.to_datetime(pd.to_datetime(tradingRecords['進場時間']).dt.date)
    tradingRecords.columns = ['id', 'date']
    tradingRecords = tradingRecords.drop_duplicates().reset_index(drop=True)
    tradingRecords['id'] = tradingRecords['id'].apply(stock_id_preprocess)
    return tradingRecords

In [ ]:
ohlc_path = 'C:\\Users\\012480\\Documents\\Data\\twstocks\\OHLC.xlsm'
close_df = pd.read_excel(ohlc_path, sheet_name='close', header=4)
open_df = pd.read_excel(ohlc_path, sheet_name='open', header=4)
low_df = pd.read_excel(ohlc_path, sheet_name='low', header=4)
high_df = pd.read_excel(ohlc_path, sheet_name='high', header=4)

close_df = preprocess(close_df)
open_df = preprocess(open_df)
low_df = preprocess(low_df)
high_df = preprocess(high_df)

In [56]:

def get_profit_intraday_distribution(tradingRecords):
    profit_distribution = []

    for row in tradingRecords.itertuples(index=False):
        stock_id = row.id
        date = row.date

        try: 
            cur_open_df = open_df[open_df.index >= date][stock_id]
            last_close_df = close_df[close_df.index >= date - timedelta(days=1)][stock_id]
            cur_low_df = low_df[low_df.index >= date][stock_id]
            cur_close_df = close_df[close_df.index >= date][stock_id]

            profit_distribution.append([stock_id, date, (cur_low_df[0] - cur_open_df[0]) / cur_open_df[0] * 100, 
            (cur_close_df[0] - cur_open_df[0]) / cur_open_df[0] * 100, 
            (cur_low_df[0] - last_close_df[0]) / last_close_df[0] * 100,
            (cur_open_df[0] - last_close_df[0]) / last_close_df[0] * 100])

        except:
            #raise Exception('Error in get_profit_distribution')
            continue
        
    return profit_distribution


In [57]:

gap_columns = ['code', 'date', 'cur_low-cur_open', 'cur_close-cur_open', 'cur_low-last_close', 'cur_open-last_close']
gap_stat = []

for strategy_report_path in tqdm(strategy_report_paths):
    strategy_name = strategy_report_path.split('\\')[-1][:-5]
    tradingRecord = preprocess_tradingRecords(strategy_report_path)
    profit_distribution = get_profit_intraday_distribution(tradingRecord)
    strategy_gap_df = pd.DataFrame(profit_distribution, columns=gap_columns)
    gap_stat.append([strategy_name, strategy_gap_df['cur_open-last_close'].median(), strategy_gap_df['cur_close-cur_open'].median(), strategy_gap_df['cur_low-cur_open'].median(), strategy_gap_df['cur_low-last_close'].median() ])


100%|██████████| 18/18 [04:24<00:00, 14.70s/it]


In [58]:
pd.DataFrame(gap_stat, columns=['strategy', 'cur_open-last_close_median', 'cur_close-cur_open_median', 'cur_low-cur_open_median', 'cur_low-last_close_median'])

,strategy,cur_open-last_close_median,cur_close-cur_open_median,cur_low-cur_open_median,cur_low-last_close_median
0,Win多2_玄哥(一日隔沖1雞酒1-1),0.000000,0.589112,-3.240122,-3.836455
1,Win多2_玄哥(一日隔沖1雞酒1-2),0.340508,0.000000,-3.025015,-3.133159
2,Win多2_玄哥(一日隔沖1雞酒1-3),1.315178,1.413416,-2.233387,-1.104110
3,Win多2_玄哥(一日隔沖2雞酒2-1),0.775982,0.354698,-2.166065,-1.664499
4,Win多2_玄哥(一日隔沖2雞酒2-2),0.638441,0.271555,-1.654944,-1.265681
5,Win多3_玄哥(三日波段1即將創高),0.194024,0.000000,-3.242497,-3.439803
6,Win多3_玄哥(三日波段2投信突買),1.662033,1.352915,-1.732558,-0.732739
7,Win多4_玄哥(五日波段1籌碼),1.141059,0.000000,-2.475218,-1.311895
8,Win多5_玄哥(十日波段1中大),0.204899,-0.470405,-1.386091,-1.172059
9,內部人10日多,0.512767,-0.505548,-2.099640,-1.667596


In [28]:
strategy_gap_df['cur_open-cur_low'].describe()

count    110.000000
mean       3.552212
std        2.964464
min        0.000000
25%        1.597085
50%        3.240122
75%        4.881555
max       17.747440
Name: cur_open-cur_low, dtype: float64

In [26]:
fig = go.Figure()

fig.add_trace(go.Bar(y=strategy_gap_df[exit_day], name=exit_day))

fig.update_layout(
    title_text=get_strategy_name(strategy_report_path),
    yaxis=dict(title='Return'),
    xaxis=dict(title='Expected return for holding T days')
)
fig.show()

NameError: name 'exit_day' is not defined

In [33]:
import plotly.graph_objects as go

def get_strategy_name(strategy_report_path):
    return strategy_report_path.split('\\')[-1][:-5]

'''
def plot_profit_distribution(profit_distribution, strategy_report_path):
    
    fig = go.Figure(
        data=[go.Bar(x=list(profit_distribution.keys()) ,y= list(profit_distribution.values()))],
        layout_title_text = get_strategy_name(strategy_report_path)
    )
    fig.show()
'''

def plot_profit_distribution(profit_distribution, strategy_report_path):
    
    fig = go.Figure()
    for exit_day in profit_distribution:
        fig.add_trace(go.Box(y=profit_distribution[exit_day], name=exit_day))

    fig.update_layout(
        title_text=get_strategy_name(strategy_report_path),
        yaxis=dict(title='Return'),
        xaxis=dict(title='Expected return for holding T days')
    )
    fig.show()


In [34]:

def get_profit_distribution(tradingRecords):
    profit_distribution = {}
    cumulative_profit_distribution = {}
    total_profit_distribution = {}
    total_cumulative_profit_distribution = {}

    for row in tradingRecords.itertuples(index=False):
        stock_id = row.id
        date = row.date
        
        if stock_id not in profit_distribution:
            profit_distribution[stock_id] = {}
            cumulative_profit_distribution[stock_id] = {}
            
        cur_open_df = open_df[open_df.index >= date][stock_id]
        cur_close_df = close_df[close_df.index >= date][stock_id]
        
        exit_days_search = min(exit_days+1, len(cur_open_df))
        
        try: 
            init_entry = cur_open_df[0]

            for exit_day in range(exit_days_search):
                if exit_day == 0:
                    entry = cur_open_df[exit_day]
                    exit = cur_close_df[exit_day]
                else:
                    entry = cur_open_df[exit_day-1]
                    exit = cur_open_df[exit_day]
                    
                profit = (exit - entry) / entry * 100 
                cumulative_profit = (exit - init_entry) / init_entry * 100 
                
                if not np.isnan(profit):
                    if exit_day in profit_distribution[stock_id]:
                        profit_distribution[stock_id][exit_day].append(profit)
                        cumulative_profit_distribution[stock_id][exit_day].append(cumulative_profit)
                    else:
                        profit_distribution[stock_id][exit_day] = [profit]
                        cumulative_profit_distribution[stock_id][exit_day] = [cumulative_profit]

                    if exit_day in total_profit_distribution:
                        total_profit_distribution[exit_day].append(profit)
                        total_cumulative_profit_distribution[exit_day].append(cumulative_profit)
                    else:
                        total_profit_distribution[exit_day] = [profit]
                        total_cumulative_profit_distribution[exit_day] = [cumulative_profit]
        except:
            #raise Exception('Error in get_profit_distribution')
            continue
        
    return total_profit_distribution, profit_distribution, total_cumulative_profit_distribution, cumulative_profit_distribution


In [77]:

def plot_profit_distribution_lines(profit_distribution, strategy_name):

    profit_distribution_list = []

    for stock_id in profit_distribution:
        cur_profit_distribution = profit_distribution[stock_id]
        for exit_day in cur_profit_distribution:
            for profit in cur_profit_distribution[exit_day]:
                profit_distribution_list.append([stock_id, exit_day, profit])
    
    profit_distribution_df = pd.DataFrame(profit_distribution_list, columns=['stock_id', 'days', 'profit']) 
    
    fig = go.Figure(data=go.Scatter(x=profit_distribution_df['days'],
                                y=profit_distribution_df['profit'],
                                mode='lines',
                                #marker_color=profit_distribution_df['profit']
                                    )
                    ) # hover text goes here
    fig.update_layout(
        title_text=strategy_name,
        yaxis=dict(title='Return'),
        xaxis=dict(title='Expected return for holding T days')
    )
    fig.show()


In [44]:


def get_profit_distribution_analysis(profit_distribution):

    profit_distribution_analysis = []

    for exit_day in profit_distribution:
        cur_distribution = profit_distribution[exit_day]
        
        profit_distribution_analysis.append([exit_day] + [round(np.quantile(cur_distribution, q), 2) for q in [0.1, 0.3]] +
         [round(np.mean(cur_distribution), 2)] + [round(np.quantile(cur_distribution, q), 2) for q in [0.5, 0.7, 0.9]] )
    
    return pd.DataFrame(profit_distribution_analysis, columns = ['holding_periods', 'quantile_10', 'quantile_30', 'mean', 'median', 'quantile_70', 'quantile_90'])


def plot_profit_distribution_analysis(profit_distribution_analysis, strategy_name):

    fig = go.Figure()

    for profit_column in profit_distribution_analysis.columns[1:]:
        if profit_column in ['mean']:
            fig.add_trace(go.Scatter(x=profit_distribution_analysis['holding_periods'], y=profit_distribution_analysis[profit_column],
                                mode='lines+markers',
                                marker=dict(symbol="diamond"),
                                line = dict(dash='dash'),
                                name=profit_column))
        else:
            fig.add_trace(go.Scatter(x=profit_distribution_analysis['holding_periods'], y=profit_distribution_analysis[profit_column],
                                mode='lines+markers',
                                name=profit_column))
                           
    fig.update_layout(
        title_text=strategy_name,
        yaxis=dict(title='Return'),
        xaxis=dict(title='Expected return for holding T days')
    )
    fig.show()


In [45]:
def plot_profit_distribution_violin(profit_distribution, strategy_name):
    
    fig = go.Figure()
    for exit_day in profit_distribution:
        fig.add_trace(go.Violin(y=profit_distribution[exit_day], name = exit_day, meanline_visible=True))

    fig.update_layout(
        title_text=strategy_name,
        yaxis=dict(title='Return'),
        xaxis=dict(title='Expected return for holding T days')
    )
    fig.show()

In [79]:

for strategy_report_path in strategy_report_paths:

    tradingRecord = preprocess_tradingRecords(strategy_report_path)
    total_profit_distribution, profit_distribution, total_cumulative_profit_distribution, cumulative_profit_distribution = get_profit_distribution(tradingRecord)
    #plot_profit_distribution_lines(cumulative_profit_distribution, get_strategy_name(strategy_report_path))
    plot_profit_distribution_violin(total_cumulative_profit_distribution, get_strategy_name(strategy_report_path))
    
    profit_distribution_analysis = get_profit_distribution_analysis(total_cumulative_profit_distribution)
    plot_profit_distribution_analysis(profit_distribution_analysis, get_strategy_name(strategy_report_path))


In [ ]:
def plot_profit_mean_distribution(profit_distribution):
    
    profit_mean_distribution = {}

    for day in profit_distribution:
        profit_mean_distribution[day] = np.mean(profit_distribution[day])

    fig = go.Figure(
        data=[go.Bar(x=list(profit_mean_distribution.keys()) ,y= list(profit_mean_distribution.values()))],
        #layout_title_text = 'TCRI'
    )
    fig.show()


In [62]:
tcri_strategy_path = 'C:\\Users\\012480\\Documents\\backtesting\\TCRI_strategy\\'

def plot_tcri_profit(pre_tcri, cur_tcri):
    
    tradingRecord = pd.read_excel(tcri_strategy_path + f'{str(pre_tcri)}to{str(cur_tcri)}.xlsx')
    tradingRecord['id'] = tradingRecord['id'].astype(str)
    print(f'TCRI {str(pre_tcri)} to {str(cur_tcri)}')
    print('Sample Size : ', len(tradingRecord))
        
    total_profit_distribution, profit_distribution, total_cumulative_profit_distribution, cumulative_profit_distribution = get_profit_distribution(tradingRecord)
    #plot_profit_distribution_marker(cumulative_profit_distribution, strategy_report_path)
    plot_profit_distribution_violin(total_cumulative_profit_distribution, f'TCRI {str(pre_tcri)} to {str(cur_tcri)}, Sample size : {len(tradingRecord)}')

    profit_distribution_analysis = get_profit_distribution_analysis(total_cumulative_profit_distribution)
    plot_profit_distribution_analysis(profit_distribution_analysis, f'TCRI {str(pre_tcri)} to {str(cur_tcri)}, Sample size : {len(tradingRecord)}')


In [65]:
tradingRecord = pd.read_excel(tcri_strategy_path + f'{str(7)}to{str(6)}.xlsx')

In [66]:
tradingRecord

,id,date,tcri,category,event,content
0,3455,2018-05-23,6(2018/03),F_市場交易,FT02_TCRI調升等級,2018/05/23 TCRI依2017/12財報調升1等，歸第6等，調等因：
1,1336,2018-06-12,6(2018/03),F_市場交易,FT02_TCRI調升等級,"新經營者入主,調整產品結構,營收大增,稅後損益轉盈,故予以調升1等."
2,1526,2018-06-26,6(2018/03),M_經營層,MT02_董監異動,董事長徐義雄續任董事，董事長尚未選任
3,4943,2018-07-02,6(2018/03),I_產業前景,IF02_投資/併購/組織策略,1.為強化聲學產品線模組化，提升經營績效，擬與啟弘股份有限公司（以下稱啟弘公司）進行策略聯盟...
4,4760,2018-09-18,6(2018/06),F_市場交易,FT02_TCRI調升等級,隨綜合評分歸第6等。
...,...,...,...,...,...,...
116,3289,2022-12-19,6(2022/09),F_市場交易,FT02_TCRI調升等級,升。
117,2520,2022-12-20,6(2022/09),F_市場交易,FT02_TCRI調升等級,打房政策對營運影響仍待觀察，僅調升 1等。
118,6174,2022-12-20,6(2022/09),F_市場交易,FT02_TCRI調升等級,景氣波動衝擊，故隨綜合評分予以調升。
119,5522,2022-12-20,6(2022/09),F_市場交易,FT02_TCRI調升等級,察，僅調升 1等。


In [167]:

entry_cost = 0.004
exit_cost = 0.007

def get_profit_distribution_R(tradingRecords):
    
    r_ratio_distribution = {}

    for row in tradingRecords.itertuples(index=False):
        stock_id = row.id
        date = row.date
        
        cur_high_df = high_df[high_df.index >= date][stock_id]
        cur_low_df = low_df[low_df.index >= date][stock_id]
        
        entry = open_df[open_df.index >= date][stock_id][0] * (1 + entry_cost)

        for exit_day in range(exit_days+1):

            exit_h = max(cur_high_df[:exit_day+1]) * (1 - exit_cost)
            exit_l = min(cur_low_df[:exit_day+1]) * (1 - exit_cost)
            profit = (exit_h - entry)
            loss = (entry - exit_l) 
            r_ratio = profit / loss
            
            if not np.isnan(profit) and not np.isnan(loss):
                if exit_day in r_ratio_distribution:
                    r_ratio_distribution[exit_day].append(r_ratio)
                    
                else:
                    r_ratio_distribution[exit_day] = [r_ratio]
                    
    return r_ratio_distribution


In [168]:
r_ratio_distribution = get_profit_distribution_R(tradingRecords)

In [169]:
plot_profit_distribution(r_ratio_distribution, strategy_report_path)

In [170]:

r_ratio_stat = []

for exit_day in r_ratio_distribution:
    r_ratio_stat.append([exit_day, np.mean(r_ratio_distribution[exit_day]), np.mean([r_ratio for r_ratio in r_ratio_distribution[exit_day] if r_ratio > 0]), np.mean([r_ratio for r_ratio in r_ratio_distribution[exit_day] if r_ratio < 0])])
    

In [171]:
r_ratio_stat

[[0, 0.23917490401726976, 1.1585583488931588, -0.3458872881764778],
 [1, 0.46276315464294615, 1.2991875358200284, -0.28561655483128534],
 [2, 0.6906994455662533, 1.4101986182643418, -0.260896234453799],
 [3, 1.0264697581045459, 1.9205990819959757, -0.22531129534345615],
 [4, 1.091542404562581, 1.824257752134847, -0.20480013344988948],
 [5, 1.117312228862849, 1.8217982240606956, -0.20712144210910274],
 [6, 1.1925744627120172, 1.8867648771391454, -0.19580636614223904],
 [7, 1.2433802667656773, 1.8637899786536136, -0.16664180570690446],
 [8, 1.330371818900132, 1.8213419588792572, -0.14253860103724328],
 [9, 1.3862916855047376, 1.8191091228665255, -0.1285693452615194],
 [10, 1.4487979742545207, 1.8647015922223704, -0.13163577402330873],
 [11, 1.4923795548869327, 1.8842366499877587, -0.13102841053077682],
 [12, 1.513050228308748, 1.8740651123618743, -0.12540193777851993],
 [13, 1.5689924388802, 1.941993635174746, -0.12385914430274148],
 [14, 1.6315607417588076, 2.0177118785804713, -0.120971

In [157]:
np.mean([r_ratio for r_ratio in r_ratio_distribution[exit_day] if r_ratio > 0])

[19.377449850126904,
 2.030346079475949,
 4.243339619406811,
 4.48737674720988,
 1.4378331233646735,
 1.0971950280497171,
 0.782011129804581,
 0.4760719925569202,
 0.050179067885557094,
 2.0748892382449706,
 0.03561010262843637,
 0.6689959105887947,
 1.7676144814937869,
 4.860427684525218,
 3.381437809052847,
 0.7242576836256132,
 0.10809028658382218,
 0.5380335016062358,
 0.6582443912001814,
 0.749266001174403,
 4.451474830185113,
 2.581702918351731,
 1.7810915146468898,
 0.0703349220160259,
 0.11073825503355765,
 0.11222244055806196,
 0.19072906673975895,
 1.5663790809927376,
 3.5769343924683845,
 0.666552237098066,
 2.949151338683377,
 0.40578198345876476,
 2.3450666406288208,
 7.557893092405008,
 4.92804775869291,
 0.27323387991616077,
 0.22191180353842183,
 0.3491612813321942,
 1.9076652785007855,
 3.6963775271555357,
 4.654231297859817,
 0.5533828705514241,
 0.8504158881321188,
 1.3980542330780372,
 10.95235212421207,
 3.1129839498360563,
 9.40697700939695,
 2.6594716186465046,
 

In [155]:
r_ratio_distribution

{0: [5.410883098916321,
  -0.08842142271182203,
  -0.04666552374421351,
  -0.24117377349839855,
  -0.11972962490996951,
  -0.35954265611257424,
  -0.37426078526418166,
  -0.17174173539155657,
  0.015475159395188919,
  0.24548269509661785,
  0.7030155781049056,
  -0.31196951325134614,
  -0.2861495275914017,
  -0.1590514981650964,
  3.8878634056298997,
  -0.3150491842610379,
  0.11222426184624147,
  1.0549551621062478,
  -0.39566375047547064,
  -0.34203551550491124,
  -0.52874576271186,
  -0.5573551263001478,
  0.2558691012568201,
  0.9721946375372492,
  -0.6038695522090327,
  -0.35321485065599345,
  -0.08352561144439159,
  -0.38801294232448374,
  -0.46587776507765943,
  0.41928106910597773,
  -0.5261839436955791,
  -0.3222263588979838,
  -0.469002887597589,
  -0.43908868386367744,
  -0.2339276530087409,
  -0.57852292020373,
  -0.39349519010536355,
  -0.46132982021078456,
  0.7032590051458073,
  1.3072901539355328,
  -0.391658812441101,
  -0.4358355230452318,
  -0.4080917964384023,
  -0.

In [135]:

strategy_report_path = strategy_report_paths[0]
tradingRecords = preprocess_tradingRecords(strategy_report_path)
total_profit_distribution, profit_distribution = get_profit_distribution(tradingRecords) 
plot_profit_distribution(total_profit_distribution, strategy_report_path)


In [82]:

for strategy_report_path in strategy_report_paths:

    tradingRecords = preprocess_tradingRecords(strategy_report_path)
    profit_distribution = get_profit_distribution(tradingRecords) 
    plot_profit_distribution(profit_distribution, strategy_report_path)


./strategy\A+_勝率73趴的五步驟選股法.xlsx


./strategy\A+_本業低本益比且投信買進.xlsx


./strategy\Win空1.xlsx


./strategy\不會寫程式.xlsx


./strategy\低市銷率關鍵券商買超.xlsx


...

./strategy\選股轉交易架構7(V13)空2.xlsx


./strategy\選股轉交易架構7(V13)空4.xlsx


./strategy\選股轉交易架構7(V13)空6.xlsx


./strategy\選股轉交易架構7.xlsx


./strategy\阿維聯集空_交易腳本.xlsx
